In [18]:
from urllib.parse import urlencode
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [19]:
def get_name_score(xing, name):
    url = "http://life.httpcn.com/xingming.asp"

    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {
        "isbz": 0,
        "xing": xing.encode("gbk"),
        "ming": name.encode("gbk"),
        "sex": 1,
        "data_type": 0,
        "year": 1980,
        "month": 2,
        "day": 25,
        "hour": 16,
        "minute": 10,
        "pid": "山西".encode("gbk"),
        "cid": "晋城".encode("gbk"),
        "wxxy": 0,
        "xishen": "金".encode("gbk"),
        "yongshen": "金".encode("gbk"),
        "check_agree": 'agree',
        "act": "submit",

    }

    params_data = urlencode(data)
    # print(params_data)
    r = requests.post(url, data=params_data, headers=headers)
    r.encoding = 'gb2312'

    if r.status_code != 200:
        raise Exception()
    # print(r.text)
    soup = BeautifulSoup(r.text, "html.parser")
    for node in soup.find_all("div", class_="chaxun_b"):
        # print(node)
        if "姓名五格评分" not in node.get_text():
            continue
        score_fonts = node.find_all("font")
        wuge_score = score_fonts[0].get_text()
        # bazi_score = score_fonts[1].get_text()
        return wuge_score.replace("分", "").strip()#, bazi_score.replace("分", "").strip()

In [20]:
first_name = "王"
datas = []
if __name__ == '__main__':
    with open("./dict/names_boys_double.txt") as fin:
        for last_name in fin:
            last_name = last_name.strip()
            if not last_name or len(last_name) == 0:
                continue
            wuge = get_name_score(first_name,last_name)
            datas.extend([{
                "name": "%s%s" % (first_name,last_name),
                "wuge":wuge
            }])

ChunkedEncodingError: ("Connection broken: TimeoutError(10060, '由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。', None, 10060, None)", TimeoutError(10060, '由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。', None, 10060, None))

In [21]:
df = pd.DataFrame(datas)
df  = df.sort_values(by="wuge", ascending=False)
df.to_csv("./output_%s.csv" % first_name, index=False,encoding="utf-8-sig")

In [ ]:

# first_name = "王"
# with open("./dict/names_boys_double.txt") as fin, open("output.txt", "w") as fout:
#     for line in fin:
#         line = line.strip()
#         if not line or len(line) == 0:
#             continue
#         wuge, bazi = get_name_score(first_name,line)
#         fout.write(
#             "\t".join([
#                 "%s%s" % (first_name,line),
#                 wuge,
#                 bazi
#             ]) + "\n"
#         )
#         fout.flush()
    